# Lab | Revisiting Machine Learning Case Study

- In this lab, you will use `learningSet.csv` file which you already have cloned in today's activities. 

### Instructions

Complete the following steps on the **categorical columns** in the dataset:

- Check for null values in all the columns
- Exclude the following variables by looking at the definitions. Create a new empty list called `drop_list`. We will append this list and then drop all the columns in this list later:
    - `OSOURCE` - symbol definitions not provided, too many categories
    - `ZIP CODE` - we are including state already
- Identify columns that over 85% missing values
- Remove those columns from the dataframe
- Reduce the number of categories in the column `GENDER`. The column should only have either "M" for males, "F" for females, and "other" for all the rest
    - Note that there are a few null values in the column. We will first replace those null values using the code below:

    ```python
    print(categorical['GENDER'].value_counts())
    categorical['GENDER'] = categorical['GENDER'].fillna('F')
    ```


In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('learningSet.csv')
display(data.head(), data.shape)

/var/folders/w7/7cfd14hn02g0d126w5q47rqw0000gn/T/ipykernel_3243/4026864892.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('learningSet.csv')


,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A


(95412, 481)

#### Standard column names

In [3]:
#cols = []

#for c in data.columns:
#    cols.append(c.lower().replace(" ", "_"))

#data.columns = cols
#data.columns

#### Spliting numerical-categorical columns

In [4]:
data.dtypes

ODATEDW       int64
OSOURCE      object
TCODE         int64
STATE        object
ZIP          object
             ...   
MDMAUD_R     object
MDMAUD_F     object
MDMAUD_A     object
CLUSTER2    float64
GEOCODE2     object
Length: 481, dtype: object

In [5]:
cat_data = data.select_dtypes(include = 'object')

display(cat_data.head(), cat_data.shape)

,OSOURCE,STATE,ZIP,MAILCODE,PVASTATE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,...,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,GRI,IL,61081,,,0,,,,,...,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,BOA,CA,91326,,,0,,,,,...,N1E,N1E,,F1E,L,G,X,X,X,A
2,AMH,NC,27017,,,0,,,,,...,,S4D,S4D,S3D,L,E,X,X,X,C
3,BRY,CA,95953,,,0,,,,,...,A1D,A1D,,,L,E,X,X,X,C
4,,FL,33176,,,0,X,X,,,...,A3D,I4E,A3D,A3D,L,F,X,X,X,A


(95412, 74)

#### Check for null values in all the columns

In [6]:
cat_data.isna().sum()

OSOURCE       0
STATE         0
ZIP           0
MAILCODE      0
PVASTATE      0
           ... 
RFA_2A        0
MDMAUD_R      0
MDMAUD_F      0
MDMAUD_A      0
GEOCODE2    132
Length: 74, dtype: int64

In [7]:
# Filtering out columns without NaN values
nan_counts = cat_data.isna().sum()
nan_counts = nan_counts[nan_counts > 0]

nan_counts

GEOCODE2    132
dtype: int64

#### Exclude the following variables by looking at the definitions.

Create a new empty list called drop_list. We will append this list and then drop all the columns in this list later:

    - OSOURCE - symbol definitions not provided, too many categories
    - ZIP CODE - we are including state already

In [8]:
# Finding columns with 'zip' in their title
zip_cols = [col for col in data.columns if 'ZIP' in col.lower()]
print(zip_cols)

[]


In [9]:
drop_list = ['OSOURCE', 'ZIP']
drop_list

['OSOURCE', 'ZIP']

#### Identify columns that over 85% missing values --> only categorical?

In [10]:
# Columns with empty rows
cat_data.columns[cat_data.isna().any()].tolist()

# The .tolist() method is used to convert an array or a pandas Series into a Python list.

['GEOCODE2']

In [11]:
# Calculating percentages
missing_percent = pd.DataFrame(cat_data.isna().sum() / len(cat_data) * 100, columns=['percent_missing']).reset_index().rename(columns={'index': 'column_name'})
missing_percent.columns = ['column_name', 'missing_fraction']
#missing_percent = cat_data.isna().mean() * 100
missing_percent

,column_name,missing_fraction
0,OSOURCE,0.000000
1,STATE,0.000000
2,ZIP,0.000000
3,MAILCODE,0.000000
4,PVASTATE,0.000000
...,...,...
69,RFA_2A,0.000000
70,MDMAUD_R,0.000000
71,MDMAUD_F,0.000000
72,MDMAUD_A,0.000000


In [12]:
# Selecting all columns with >85% of missing values --> none in cat, multiple in data
columns_above_threshold = missing_percent[missing_percent['missing_fraction']>0.85]
print(len(columns_above_threshold['column_name']))

#missing_percent[missing_percent > 85]

0


#### Remove those columns from the dataframe

In [13]:
# There is none in categorical.

#### Reduce the number of categories in the column `GENDER`.
The column should only have either "M" for males, "F" for females, and "other" for all the rest.


Note that there are a few null values in the column. We will first replace those null values using the code below:

    ```python
    print(categorical['GENDER'].value_counts())
    categorical['GENDER'] = categorical['GENDER'].fillna('F')
    ```

In [14]:
cat_data['GENDER'].value_counts()

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [15]:
# Filling null values as indicated
cat_data['GENDER'] = cat_data['GENDER'].fillna('F')

cat_data['GENDER'].value_counts()

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [16]:
# Reducing number of categories
cat_data['GENDER'] = cat_data["GENDER"].replace({" ": "other", "U": "other", "J": "other", "C": "other", "A": "other", "" : "other"})

# print the value counts to verify the result
cat_data["GENDER"].value_counts()

F        51277
M        39094
other     5041
Name: GENDER, dtype: int64

# Lab | Feature engineering

In this lab, you will use learningSet.csv file which you have already cloned in the previous activities.

#### Instructions
Here we will work on cleaning some of the other columns in the dataset using the techniques that we used before in the lessons.

    - Check for null values in the numerical columns.
    - Use appropriate methods to clean the columns GEOCODE2, WEALTH1, ADI, DMA,and MSA.
    - Use appropriate EDA technique where ever necessary.

#### Selecting numerical columns

In [18]:
num_data = data.select_dtypes(np.number)

num_data.head()

,ODATEDW,TCODE,DOB,AGE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,...,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2F,CLUSTER2
0,8901,0,3712,60.0,NaN,NaN,NaN,0,NaN,NaN,...,8911,9003.0,4.0,7.741935,95515,0,0.0,0,4,39.0
1,9401,1,5202,46.0,1.0,6.0,9.0,16,0.0,0.0,...,9310,9504.0,18.0,15.666667,148535,0,0.0,0,2,1.0
2,9001,1,0,NaN,NaN,3.0,1.0,2,0.0,0.0,...,9001,9101.0,12.0,7.481481,15078,0,0.0,1,4,60.0
3,8701,0,2801,70.0,NaN,1.0,4.0,2,0.0,0.0,...,8702,8711.0,9.0,6.812500,172556,0,0.0,1,4,41.0
4,8601,0,2001,78.0,1.0,3.0,2.0,60,1.0,0.0,...,7903,8005.0,14.0,6.864865,7112,0,0.0,1,2,26.0


#### Check for null values in the numerical columns.

In [19]:
num_data.isna().sum()

ODATEDW         0
TCODE           0
DOB             0
AGE         23665
NUMCHLD     83026
            ...  
TARGET_B        0
TARGET_D        0
HPHONE_D        0
RFA_2F          0
CLUSTER2      132
Length: 407, dtype: int64

In [21]:
# Filtering out columns without NaN values
nan_counts = num_data.isna().sum()
nan_counts = nan_counts[nan_counts > 0]

nan_counts

AGE         23665
NUMCHLD     83026
INCOME      21286
WEALTH1     44732
MBCRAFT     52854
            ...  
RAMNT_23    87553
RAMNT_24    77674
NEXTDATE     9973
TIMELAG      9973
CLUSTER2      132
Length: 91, dtype: int64

#### Use appropriate methods to clean the columns GEOCODE2, WEALTH1, ADI, DMA,and MSA.

- Geocode

In [25]:
#### Geocode2 is CATEGORICAL
cat_data['GEOCODE2']

0        C
1        A
2        C
3        C
4        A
        ..
95407    C
95408    A
95409    B
95410    A
95411    C
Name: GEOCODE2, Length: 95412, dtype: object

In [27]:
cat_data['GEOCODE2'].value_counts()

A    34484
B    28505
D    16580
C    15524
       187
Name: GEOCODE2, dtype: int64

In [28]:
# Adding " " to biggest value count ('A')
cat_data['GEOCODE2'] = cat_data["GEOCODE2"].replace({" ": "A"})
cat_data['GEOCODE2'].value_counts()


A    34671
B    28505
D    16580
C    15524
Name: GEOCODE2, dtype: int64

- Wealth1

In [29]:
num_data['WEALTH1'].value_counts()

9.0    7585
8.0    6793
7.0    6198
6.0    5825
5.0    5280
4.0    4810
3.0    4237
2.0    4085
1.0    3454
0.0    2413
Name: WEALTH1, dtype: int64

In [30]:
num_data['WEALTH1'].describe()

count    50680.000000
mean         5.345699
std          2.742490
min          0.000000
25%          3.000000
50%          6.000000
75%          8.000000
max          9.000000
Name: WEALTH1, dtype: float64

In [ ]:
# Nothing to do

- ADI

In [31]:
num_data['ADI'].value_counts()

13.0     7296
51.0     4622
65.0     3765
57.0     2836
105.0    2617
         ... 
651.0       1
103.0       1
601.0       1
161.0       1
147.0       1
Name: ADI, Length: 204, dtype: int64

In [32]:
num_data['ADI'].describe()

count    95280.000000
mean       187.356402
std        137.019184
min          0.000000
25%         65.000000
50%        175.000000
75%        279.000000
max        651.000000
Name: ADI, dtype: float64

In [ ]:
# ADI code - no extra information provided in docs --> nothing to do

- DMA

In [33]:
num_data['DMA'].value_counts()

803.0    7296
602.0    4632
807.0    3765
505.0    2839
819.0    2588
         ... 
569.0       1
554.0       1
584.0       1
552.0       1
516.0       1
Name: DMA, Length: 206, dtype: int64

In [34]:
num_data['ADI'].describe()

count    95280.000000
mean       187.356402
std        137.019184
min          0.000000
25%         65.000000
50%        175.000000
75%        279.000000
max        651.000000
Name: ADI, dtype: float64

- MSA

In [35]:
num_data['DMA'].value_counts()

803.0    7296
602.0    4632
807.0    3765
505.0    2839
819.0    2588
         ... 
569.0       1
554.0       1
584.0       1
552.0       1
516.0       1
Name: DMA, Length: 206, dtype: int64

In [36]:
num_data['MSA'].describe()

count    95280.000000
mean      3527.744102
std       2863.904737
min          0.000000
25%        520.000000
50%       3350.000000
75%       5960.000000
max       9360.000000
Name: MSA, dtype: float64

#### Use appropriate EDA technique where ever necessary.